In [2]:
import json
import pandas as pd
from elasticsearch import Elasticsearch
import mariadb
from pyhive import hive
from SPARQLWrapper import SPARQLWrapper, JSON
from pyhive import hive

# **1-GRAPHDB** Descubrimiento

In [104]:
# Set up the SPARQL endpoint URL
sparql = SPARQLWrapper("http://localhost:8000/repositories/GESTBD")

sparql.setQuery("""
select  ?endpoint
WHERE{
    ?instance <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://upm.es/ont/Relational_Database> .
	 ?instance <http://upm.es/ont/endpoint> ?endpoint .
}""")
sparql.setReturnFormat(JSON)

# Execute the query and print results
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    for key, value in result.items():
        for key1, value1 in value.items():
            if value1 != 'literal':
                endpoint_rl=value1
endpoint_rl

URLError: <urlopen error [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión>

In [ ]:
sparql.setQuery("""
    select  ?index_name ?mapping_field
WHERE{
    ?instance <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://upm.es/ont/Relational_Database> .
    ?instance <http://upm.es/ont/topic> "F1".
	 ?instance <http://upm.es/ont/contains_table> ?object .
    ?object <http://upm.es/ont/contains_column> ?a.
    ?object <http://upm.es/ont/name> ?index_name.
    ?a <http://upm.es/ont/name> ?mapping_field.
}
""")

# Specify the return format (JSON)
sparql.setReturnFormat(JSON)
columns_rl=[]
# Execute the query and print results
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    for key, value in result.items():

        for key1, value1 in value.items():
            if value1 != 'literal':
                columns_rl.append(value1) 

columns_rl


['hive_constructors',
 'name',
 'hive_drivers',
 'forename',
 'hive_drivers',
 'driverId',
 'hive_drivers',
 'number',
 'hive_drivers',
 'surname',
 'hive_races',
 'name',
 'hive_races',
 'year',
 'hive_results',
 'position']

In [64]:
sparql.setQuery("""
select  ?endpoint 
WHERE{
    ?instance <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://upm.es/ont/Elasticsearch_Database> .
	 ?instance <http://upm.es/ont/endpoint> ?endpoint .
}""")
sparql.setReturnFormat(JSON)

# Execute the query and print results
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    for key, value in result.items():
        for key1, value1 in value.items():
            if value1 != 'literal':
                endpoint_elastic=value1
endpoint_elastic



'http://localhost:9200'

In [57]:
sparql.setQuery("""
select  ?index_name ?mapping_field
WHERE{
    ?instance <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://upm.es/ont/Elasticsearch_Database>.
    ?instance <http://upm.es/ont/topic> "F1".
	 ?instance <http://upm.es/ont/contains_index> ?object .
    ?object <http://upm.es/ont/contains_mapping_field> ?a.
    ?object <http://upm.es/ont/name> ?index_name.
    ?a <http://upm.es/ont/name> ?mapping_field.
}""")
sparql.setReturnFormat(JSON)

# Execute the query and print results
results = sparql.query().convert()
columns_elastic=[]
for result in results["results"]["bindings"]:
    for key, value in result.items():
        for key1, value1 in value.items():
            if value1 != 'literal':
                columns_elastic.append(value1) 
columns_elastic

EndPointNotFound: EndPointNotFound: It was not possible to connect to the given endpoint: check it is correct. 

Response:
b'Unknown repository: GESTBD'

# **1º-Consulta**

Vamos a sacar el piloto con mayor número de victorias en 2023, y de ahí su ranking medio

In [3]:
#Uses Postgres and MariaDB 
conn = hive.Connection(host="localhost", username="hive", database="f1_hive")
cursor = conn.cursor()

In [ ]:
cursor.execute('''
SELECT d.forename, d.surname , COUNT(*) AS victorias
FROM hive_results r
JOIN hive_races ra ON r.raceid = ra.raceid
JOIN hive_drivers d ON r.driverid = d.driverid
WHERE r.position = 1 AND ra.year = 2023
GROUP BY d.forename, d.surname
ORDER BY victorias DESC
LIMIT 1
''')
results = cursor.fetchall()
print(results[0])
name_driver= results[0][0] +' '+ results[0][1]
print(name_driver)
cursor.close()
conn.close()

('Max', 'Verstappen', 19)
Max Verstappen


In [ ]:
es = Elasticsearch("http://localhost:9200")
# Consulta en Elasticsearch para obtener la posición media del piloto en 2023
res = es.search(index='my_index', body={
    "size": 0,
    "query": {
        "bool": {
            "must": [
                {"term": {"pilot_name": {"value": name_driver}}},
                {"range": {"ranking_date": {"gte": "2023-01-01", "lte": "2023-12-31"}}}
            ]
        }
    },
    "aggs": {
        "posicion_media": {
            "avg": {
                "field": "position"
            }
        }
    }
})

# Procesa la respuesta de Elasticsearch
if res.get("aggregations"):
    posicion_media = res["aggregations"]["posicion_media"]["value"]
    print(f"Posición media de {name_driver} en el ranking en 2023: {posicion_media}")
else:
    print("No se encontró información de posición media en Elasticsearch para este piloto en 2023.")


Posición media de Max Verstappen en el ranking en 2023: 4.5


# **2º-Consulta**

# **2-ELASTICSEARCH**

In [98]:
es = Elasticsearch("http://localhost:9200")
res = es.search(index='my_index', body={
  'query': {
    'bool': {
      'must': [
        { 'term': { 'pilot_name': 'Lance Stroll' } }
      ],
      
    }
  },
'sort': [
    { 'position': 'asc' }  # Ordena por 'score' de mayor a menor
  ],
  "size":1
})
position=res['hits']['hits'][0]['_source']['position']

In [106]:
position

4

In [3]:
conn = hive.Connection(host="localhost", username="hive", database="f1_hive")
cursor = conn.cursor()

In [ ]:
conn = hive.Connection(host="localhost", username="hive", database="f1_hive")
cursor = conn.cursor()
cursor.execute('''
SELECT min(d.position) AS Mejor_Posicion
FROM hive_results r
JOIN hive_drivers d ON r.driverid = d.driverid
WHERE d.forename='Lance' and d.surname='Stroll'
''')
results = cursor.fetchall()
print(results[0])
cursor.close()
conn.close()